In [1]:
import torch
import sys
import numpy as np
import itertools
from models import *
from dataset import *
from torch.utils.data import DataLoader
from torch.autograd import Variable
import argparse
import time
import datetime

In [2]:
#SET PARAMS:
dataset_path = "/var/datasets/videos/UCF-101-frames"
split_path = "/var/datasets/videos/ucfTrainTestlist"
split_number =2
img_dim =112
channels = 3
latent_dim = 512
checkpoint_model = "model_checkpoints/ConvLSTM_80.pth"
sequence_length = 40 


In [4]:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    image_shape = (channels, img_dim, img_dim)
    
    # Classification criterion
    cls_criterion = nn.CrossEntropyLoss().to(device)
    
    # Define test set
    test_dataset = Dataset(
        dataset_path=dataset_path,
        split_path=split_path,
        split_number=split_number,
        input_shape=image_shape,
        sequence_length=sequence_length,
        training=False,
    )
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4)

    # Define network
    model = ConvLSTM(
        num_classes=test_dataset.num_classes,
        latent_dim=latent_dim,
        lstm_layers=1,
        hidden_dim=1024,
        bidirectional=True,
    )
    model = model.to(device)
    model.load_state_dict(torch.load(checkpoint_model))
    model.eval()

ConvLSTM(
  (encoder): Encoder(
    (feature_extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)

In [5]:
#IN SHORT:
test_accuracies = []
for batch_i, (X, y) in enumerate(test_dataloader):
    image_sequences = Variable(X.to(device), requires_grad=False)
    labels = Variable(y, requires_grad=False).to(device)
    with torch.no_grad():
        # Reset LSTM hidden state
        model.lstm.reset_hidden_state()
        # Get sequence predictions
        predictions = model(image_sequences)



    # Compute accuracy using the most common prediction for each sequence
    acc = 100 * (predictions.detach().argmax(1) == labels).cpu().numpy().mean()

    # Keep track of accuracy
    test_accuracies.append(acc)
    # Log test performance

    sys.stdout.write(
        "\rTesting -- [Batch %d/%d] [Acc: %.2f%% (%.2f%%)]"
        % (batch_i, len(test_dataloader),  acc, np.mean(test_accuracies))
    )

Testing -- [Batch 3733/3734] [Acc: 100.00% (99.14%)]